In [1]:
import numpy as np
np.random.randint(0, 255)

56

In [2]:
import cv2



def rotate_image( img, angle = 0):
        # get cv2 image
        # this function get an image and angle then rotate image by this angle


        row1 = np.hstack(( cv2.flip(cv2.flip(img,1), 0) , cv2.flip(img, 0) , cv2.flip(cv2.flip(img,1), 0) ))
        row2 = np.hstack(( cv2.flip(img, 1) , img , cv2.flip(img, 1) ))
        row3 = np.hstack(( cv2.flip(cv2.flip(img,1), 0) , cv2.flip(img, 0) , cv2.flip(cv2.flip(img,1), 0) ))
        
        img1 = np.vstack((row1 , row2))
        img1 = np.vstack((img1 , row3))


        height, width = img1.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)

        if img.shape[2] == 4:
            borderValue = (255,255,255,0)
        else:
            borderValue = (255,255,255)

        rotated_image = cv2.warpAffine(img1, rotation_matrix, (width, height) , borderValue = borderValue , flags=cv2.INTER_NEAREST)

        rotated_image = rotated_image[img.shape[0]:img.shape[0]*2  , img.shape[1]:img.shape[1]*2  , :]


        return rotated_image

In [3]:
import numpy as np

from ultralytics import RTDETR

model_yolo =  RTDETR("rtdetr-l.pt")
def detect_main_car( img):
        
        results = model_yolo.predict(img , verbose=False  , conf=.7)
        results = results[0].cpu().numpy().boxes.data

        if results.shape[0] == 0:
            return None
        
        # select cars and trucks
        indices = np.where((results[:, 5] != 2) & (results[:, 5] != 7))
        # Delete the rows where not car or truck
        cars = np.delete(results, indices, axis=0)
        
        diffs = cars[:, 3] - cars[:, 1]

        # Get the index of the row with the maximum difference (main car)
        max_index = np.argmax(diffs)

        # Get the row with the maximum difference
        biggest_car = cars[max_index]
       

        if cars.shape[0] == 0:
            return None
        dict = {
            "xmin":  int(biggest_car[0]) ,
            "ymin":  int(biggest_car[1]) ,
            "xmax":  int(biggest_car[2]) , 
            "ymax":  int(biggest_car[3]) ,
        }


        



        return dict

In [4]:
with open("text_file.txt", "a") as f:
    f.write("appended text")


In [5]:
import glob

start = 162253
c=0
for im in glob.glob("raw_image_blender/*"):
    if c%10==0:
        print(c)
    c+=1
    try:
        img = cv2.imread(im)
        bb = detect_main_car(img)
        img0 = img[bb["ymin"]:bb["ymax"] , bb["xmin"]:bb["xmax"] , :]
        cv2.imwrite("blender_ds/"+str(start)+"_ang.jpg" , img0)
        with open("angs.txt", "a") as f:
            f.write(str(start)+"_ang.jpg    0\n")
        start+=1
    except Exception as e :
        print(im , "zero" , e)
        start+=1

    for ang in range(1,31):
        try:
            imP = rotate_image(img , ang)
            bb = detect_main_car(imP)
            imP = imP[bb["ymin"]:bb["ymax"] , bb["xmin"]:bb["xmax"] , :]
            cv2.imwrite("blender_ds/"+str(start)+"_ang.jpg" , imP)
            with open("angs.txt", "a") as f:
                f.write(str(start)+"_ang.jpg    "+str(ang)+"\n")
            start+=1
        except Exception as e:
            start+=1
            print(im , ang , e)


        try:
            imP = rotate_image(img , -ang)
            bb = detect_main_car(imP)
            imP = imP[bb["ymin"]:bb["ymax"] , bb["xmin"]:bb["xmax"] , :]
            cv2.imwrite("blender_ds/"+str(start)+"_ang.jpg" , imP)
            with open("angs.txt", "a") as f:
                f.write(str(start)+"_ang.jpg    "+str(-ang)+"\n")
            start+=1
        except Exception as e :
            start+=1
            print(im , -ang , e)
            




0
10
20
raw_image_blender\1.2021 Ford Mustang MACH-E GT ORG-0021.jpg 30 attempt to get argmax of an empty sequence
30
raw_image_blender\1.2021 Ford Mustang MACH-E GT ORG-0039.jpg 22 'NoneType' object is not subscriptable
raw_image_blender\1.2021 Ford Mustang MACH-E GT ORG-0039.jpg -23 'NoneType' object is not subscriptable
40
raw_image_blender\10.BMX X7 2019.001-0007.jpg 26 OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

raw_image_blender\10.BMX X7 2019.001-0007.jpg 29 OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

raw_image_blender\10.BMX X7 2019.001-0007.jpg 30 OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

raw_image_blender\10.BMX X7